In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

x, y = make_classification(n_samples=225, n_features=1, n_classes=2, n_redundant=0,
                           n_repeated=0, n_informative=1, n_clusters_per_class=1, weights=[0.2, 0.8])

In [2]:
df = pd.DataFrame({'x':x.reshape(-1), 'y':y})
df.head()

,x,y
0,0.632761,1
1,1.108102,1
2,1.124019,1
3,1.052722,1
4,0.750665,1


In [3]:
X = x.reshape(-1)

In [4]:

a = 1
b = 1
n_iter = 50
nu = 0.1

pred = a * X + b

for i in range(n_iter):
    
    err = y - pred
    
    a_change = X * np.sign(err)
    b_change = np.sign(err)
    
    a += nu * a_change.mean()
    b += nu * b_change.mean()
    
    pred = a * X + b
    
    err = y - pred
    
    if i % 10 == 0:
        print(np.abs(err.mean()))

0.6596680196302463
0.011480991660707441
0.018411963396022406
0.018766996832098752
0.019122030268175105


In [5]:
pred_proba = np.where(pred <= 0.5, 0, 1)

pred_proba = pred_proba.reshape(-1)
pred_proba

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0])

In [6]:
print(sum(y != pred_proba))

1


--------------------------------

In [56]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn import metrics

x, y = make_classification(n_samples=100, n_features=2, n_informative=1,
                           n_clusters_per_class=1, n_classes=2, n_repeated=0, n_redundant=0, weights=[0.2, 0.8])

In [53]:

# Простая функция для тренировки слабого классификатора на основе порога
def train_weak_classifier(X, y, weights):
    n_samples, n_features = X.shape
    best_classifier = None
    min_error = float('inf')
    
    # Перебираем каждый признак
    for feature_idx in range(n_features):
        feature_values = X[:, feature_idx]
        thresholds = np.unique(feature_values)
        
        # Перебираем возможные пороги для классификации
        for threshold in thresholds:
            # Пробуем обе полярности классификации (<= threshold или > threshold)
            for polarity in [1, -1]:
                predictions = np.ones(y.shape)
                predictions[feature_values * polarity <= threshold * polarity] = -1
                
                # Ошибка классификации с учетом весов
                error = np.sum(weights[y != predictions])
           
                # Если ошибка меньше минимальной, сохраняем этот классификатор
                if error < min_error:
                    min_error = error
                    best_classifier = {
                        'feature_idx': feature_idx,
                        'threshold': threshold,
                        'polarity': polarity
                    }
    
    return best_classifier, min_error

# Функция для предсказания на основе слабого классификатора
def predict_weak_classifier(X, classifier):
    feature_values = X[:, classifier['feature_idx']]
    predictions = np.ones(feature_values.shape)
    predictions[classifier['polarity'] * feature_values <= classifier['polarity'] * classifier['threshold']] = -1
    return predictions

# Реализация алгоритма AdaBoost
def adaboost(X, y, n_classifiers):
    n_samples, n_features = X.shape
    weights = np.ones(n_samples) / n_samples
    classifiers = []
    alphas = []
    
    for _ in range(n_classifiers):
        # Тренируем слабый классификатор
        classifier, error = train_weak_classifier(X, y, weights)
        
        # Вычисляем вес (альфа) для данного классификатора
        alpha = 0.5 * np.log((1 - error) / (error + 1e-10))  # Добавляем небольшое число к ошибке для стабильности
        
        # Предсказания слабого классификатора
        predictions = predict_weak_classifier(X, classifier)
        
        # Обновляем веса образцов
        weights *= np.exp(-alpha * y * predictions)
        weights /= np.sum(weights)
        
        # Сохраняем классификатор и его вес
        classifiers.append(classifier)
        alphas.append(alpha)
    
    return classifiers, alphas

# Функция для предсказания на основе всех слабых классификаторов
def predict(X, classifiers, alphas):
    weak_predictions = np.array([alpha * predict_weak_classifier(X, clf) for clf, alpha in zip(classifiers, alphas)])
    return np.sign(np.sum(weak_predictions, axis=0))

# Пример использования
if __name__ == '__main__':
    # Пример данных: двумерный массив X и метки классов y

    # Запуск алгоритма AdaBoost
    n_classifiers = 5
    classifiers, alphas = adaboost(x, y, n_classifiers)

    # Предсказания
    predictions = predict(x, classifiers, alphas)
    print(f'Предсказания: {predictions}')


Предсказания: [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


--------

In [80]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [81]:
x, y = datasets.make_classification(n_samples=450, n_features=2, n_clusters_per_class=2, n_classes=2 
                                    ,n_informative=2, n_redundant=0, n_repeated=0, weights=[0.4, 0.6])

In [82]:
df = pd.DataFrame(x, columns=['pressure', 'weight'])
df['y'] = y

In [83]:
df = df.rename({'y': 'sport'}, axis=1)

In [84]:
df.head()

,pressure,weight,sport
0,1.179391,-2.801843,0
1,1.180264,-0.991631,0
2,0.177245,0.104921,1
3,2.103334,2.226220,1
4,-1.455742,-0.437599,0


In [85]:
x_tain, x_test, y_train, y_test = train_test_split(df.iloc[::, :-1], df['sport'], test_size=0.3, random_state=1)

In [117]:
model = AdaBoostClassifier(estimator=RandomForestClassifier(), n_estimators=10, random_state=1)
model.fit(x_tain, y_train)

AdaBoostClassifier(estimator=RandomForestClassifier(), n_estimators=10,
                   random_state=1)

In [118]:
model.score(x_tain, y_train)

1.0

----

In [119]:
model_log = LogisticRegression()
model_log.fit(x_tain, y_train)

LogisticRegression()

In [120]:
model_log.score(x_tain, y_train)

0.8984126984126984

---

In [123]:
model_deci = DecisionTreeClassifier(max_depth=3)
model_deci.fit(x_tain, y_train)

DecisionTreeClassifier(max_depth=3)

In [125]:
model_deci.score(x_tain, y_train)

0.9238095238095239

---------------------

In [126]:
np.r_

-------

In [52]:
import numpy as np
from sklearn import datasets
from sklearn import metrics

x, y = datasets.make_classification(n_samples=500, n_features=1, n_classes=2,
                                                    n_clusters_per_class=1, n_informative=1, n_redundant=0, n_repeated=0, weights=[0.38, 0.62])


In [61]:
import numpy as np

# Простая функция для тренировки слабого классификатора на основе порога
def train_weak_classifier(X, y, weights):
    n_samples, n_features = X.shape
    best_classifier = None
    min_error = float('inf')
    
    # Перебираем каждый признак
    for feature_idx in range(n_features):
        feature_values = X[:, feature_idx]
        thresholds = np.unique(feature_values)
        
        # Перебираем возможные пороги для классификации
        for threshold in thresholds:
            # Пробуем обе полярности классификации (<= threshold или > threshold)
            for polarity in [1, 0]:
                predictions = np.ones(y.shape)
                predictions[feature_values * polarity <= threshold * polarity] = 0
                
                # Ошибка классификации с учетом весов
                error = np.sum(weights[y != predictions])
                
                # Если ошибка меньше минимальной, сохраняем этот классификатор
                if error < min_error:
                    min_error = error
                    best_classifier = {
                        'feature_idx': feature_idx,
                        'threshold': threshold,
                        'polarity': polarity
                    }
    
    return best_classifier, min_error

# Функция для предсказания на основе слабого классификатора
def predict_weak_classifier(X, classifier):
    feature_values = X[:, classifier['feature_idx']]
    predictions = np.ones(feature_values.shape)
    predictions[classifier['polarity'] * feature_values <= classifier['polarity'] * classifier['threshold']] = 0
    return predictions

# Реализация алгоритма AdaBoost
def adaboost(X, y, n_classifiers):
    n_samples, n_features = X.shape
    weights = np.ones(n_samples) / n_samples
    classifiers = []
    alphas = []
    
    for _ in range(n_classifiers):
        # Тренируем слабый классификатор
        classifier, error = train_weak_classifier(X, y, weights)
        
        # Вычисляем вес (альфа) для данного классификатора
        alpha = 0.5 * np.log((1 - error) / (error + 1e-10))  # Добавляем небольшое число к ошибке для стабильности
        
        # Предсказания слабого классификатора
        predictions = predict_weak_classifier(X, classifier)
        
        # Обновляем веса образцов
        weights *= np.exp(-alpha * y * predictions)
        weights /= np.sum(weights)
        
        # Сохраняем классификатор и его вес
        classifiers.append(classifier)
        alphas.append(alpha)
    
    return classifiers, alphas

# Функция для предсказания на основе всех слабых классификаторов
def predict(X, classifiers, alphas):
    weak_predictions = np.array([alpha * predict_weak_classifier(X, clf) for clf, alpha in zip(classifiers, alphas)])
    return np.sign(np.sum(weak_predictions, axis=0))

# Пример использования
if __name__ == '__main__':
    # Пример данных: двумерный массив X и метки классов y
    
    # Запуск алгоритма AdaBoost
    n_classifiers = 10
    classifiers, alphas = adaboost(x_train, y_train, n_classifiers)

    # Предсказания
    predictions = predict(x_test, classifiers, alphas)
    print(f'Предсказания: {predictions}')


Предсказания: [1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0.
 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0.
 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1.
 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1.
 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0.
 0. 1. 1. 1. 0. 1.]


In [64]:
y_test

array([1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1])

In [66]:
metrics.accuracy_score(y_test, predictions)

0.9866666666666667

----

In [57]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

model_ada = AdaBoostClassifier(n_estimators=10, random_state=1)
model_ada.fit(x_train, y_train)

AdaBoostClassifier(n_estimators=10, random_state=1)

In [58]:
model_ada.score(x_train, y_train)

0.9885714285714285

In [59]:
y_pred = model_ada.predict(x_test)

In [60]:
print(metrics.accuracy_score(y_test, y_pred))

0.9866666666666667


In [29]:
1/8 * np.e

0.33978522855738064

In [34]:
1/8 * np.e**-0.97

0.047385379762924854